In [55]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [56]:
file_path = "weather_data.csv"

# Read the mouse data and the study results
df = pd.read_csv(file_path)
df.head()

,city,lat_long,latitude,longitude,temperature,humidity,cloudiness,wind_speed,country,timezone,Hemisphere
0,raudeberg,"(63.58902360898884, 2.867203051415771)",63.589024,2.867203,37.87,87.0,90.0,1.99,IQ,3600,Northern
1,rikitea,"(-36.31850794653735, -110.4928982258566)",-36.318508,-110.492898,78.98,72.0,10.0,17.98,IQ,-32400,Southern
2,yellowknife,"(84.90294674751334, -112.96270506552527)",84.902947,-112.962705,-7.04,90.0,0.0,0.00,IQ,-25200,Northern
3,margate,"(-34.243375269255644, 34.935160871335626)",-34.243375,34.935161,39.76,72.0,11.0,8.99,IQ,0,Southern
4,cape town,"(-65.00102316551985, -8.614653840596475)",-65.001023,-8.614654,63.57,88.0,75.0,1.99,IQ,7200,Southern


In [57]:
locations = zip(df.latitude, df.longitude)
locations = list(locations)

In [58]:
# heat map of locations
#https://jupyter-gmaps.readthedocs.io/en/latest/api.html

fig = gmaps.figure()

locations = zip(df.latitude, df.longitude)
locations = list(locations)

fig.add_layer(gmaps.heatmap_layer(locations, weights=df.humidity, dissipating=False, point_radius=5))
                                 
fig


Figure(layout=FigureLayout(height='420px'))

In [59]:
#search criteria for ideal location

mask = (df.temperature > 70) & (df.temperature < 80) & (df.wind_speed < 10) & (df.cloudiness == 0)

ideal = df.loc[mask]

ideal

,city,lat_long,latitude,longitude,temperature,humidity,cloudiness,wind_speed,country,timezone,Hemisphere
165,new norfolk,"(-49.67204873533861, 143.03219708307097)",-49.672049,143.032197,77.63,28.0,0.0,4.00,IQ,39600,Southern
281,alice town,"(24.47313012764573, -79.11813313101258)",24.473130,-79.118133,74.19,75.0,0.0,6.78,IQ,-18000,Northern
354,saint-louis,"(16.400877568243615, -16.304227871529065)",16.400878,-16.304228,73.24,12.0,0.0,9.15,IQ,0,Northern
382,san juan,"(-31.179526259868204, -67.72594125952811)",-31.179526,-67.725941,72.55,37.0,0.0,9.26,IQ,-10800,Southern
408,pochutla,"(11.013066028124129, -95.78777049488407)",11.013066,-95.787770,76.82,80.0,0.0,3.13,IQ,-21600,Northern
481,launceston,"(-41.75931461156571, 147.4738973090689)",-41.759315,147.473897,78.98,37.0,0.0,5.01,IQ,39600,Southern


# Hotels

In [62]:
# set up lists
cities = []
countries= []
lats= []
longs= []
hotels= []
ratings= []
addresses= []


# set up a parameters dictionary
params = {
    "keyword": "",
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


# Use the lat/lng we recovered to identify airports
for index, row in ideal.iterrows():
    # get lat, lng from df
    lat = ideal["latitude"]
    lng = ideal["longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
   
    # convert to json
    name_address = name_address.json()

    #grab data
    hotel = name_address["results"][0]["name"]
    address = name_address["results"][0]["vicinity"]
    rating = name_address["results"][0]["rating"]
    
    # put it all in a list
    hotels.append(hotel)
    ratings.append(rating)
    addresses.append(address)
    cities.append(row.city)
    countries.append(row.country)
    lats.append(row.latitude)
    longs.append(row.longitude)

IndexError: list index out of range

In [63]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params["location"] = f"-49.672049,143.032197"

# make request and print url
name_address = requests.get(base_url, params=params)
print(name_address)
# convert to json
name_address = name_address.json()
print(name_address)


<Response [200]>
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}


In [ ]:
# create new dataframe
hotel_df = pd.DataFrame()
hotel_df['city'] = cities
hotel_df['country'] = countries
hotel_df['latitude'] = lats
hotel_df['longitude'] = longs
hotel_df['hotel'] = hotels
hotel_df['address'] = addresses
hotel_df['rating'] = ratings

hotel_df

In [64]:
# Assign the marker layer to a variable
#markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig.add_layer(markers)
fig

NameError: name 'coordinates' is not defined

Resources used:
https://jupyter-gmaps.readthedocs.io/en/latest/api.html